# **Implémentation d'un modèle de scoring**
## **Notebook 6/6 - Assemblage des données TEST pour le dashboard**
*Sofia Chevrolat (Novembre 2020)*
___
Cette étude vise à réaliser un modèle permettant de prédire le risque de défaut de remboursement d'un prêt pour un client donné. Ce modèle doit être basé sur une variété de données (personnelles, en provenance de différentes sources bancaires, etc.).

Ce modèle est destiné à être servi via une API, elle-même appelée via un dashboard interactif. Le modèle devra donc être exporté une fois finalisé.
___
_**Remerciements**:<br>
Merci à mon compagnon [J. Duplan](https://www.linkedin.com/in/julian-duplan-64844a41/) pour les discussions intéressantes.<br>
Merci également à mon mentor [Samia Drappeau](https://www.linkedin.com/in/samiadrappeau) pour les échanges d'idées, les conseils et les encouragements !_
___


Ce notebook est organisé comme suit:

**0. Mise en place**
- 0.1 Chargement des librairies et fonctions utiles
- 0.2 Chargement et description du jeu de données
- 0.3 Assemblage du jeux de données
    
**1. Nettoyage**
- 1.1 Correction orthographique et typologiques
- 1.2 Suppression des outliers
    * 1.2.1 Suppression des produits dont le poids indiqué est <= 0
    * 1.2.2 Suppression des erreurs de workflow
- 1.3 Suppression des features les moins renseignées

**4. Analyse exploratoire**
- 4.1 Grandeurs statistiques
    * 4.1.1 Tendance centrale
        * 4.1.1.1 Features qualitatives
        * 4.1.1.2 Features quantitatives
        
**5. Export des données**

**6. Conclusion**

___
### 0. MISE EN PLACE

Dans cette première étape, le cadre de travail est mis en place, c'est-à-dire :
- Les librairies et packages Python nécessaires sont chargés
- Les fonctions utiles sont définies
- Le jeu de données est chargé
___

___
#### 0.1 CHARGEMENT DES LIBRAIRIES ET FONCTIONS UTILES

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from joblib import load, dump

In [3]:
from sys import path
path.append("./Resources/functions")

import helper_functions as hf
import graphical_functions as gf

___
#### 0.2 CHARGEMENT ET DESCRIPTION DU JEU DE DONNÉES

In [4]:
app_test = pd.read_csv("./Resources/datasets/origin/application_test.csv")

bureau_balance = pd.read_csv("./Resources/datasets/origin/bureau_balance.csv")

bureau = pd.read_csv("./Resources/datasets/origin/bureau.csv")

card = pd.read_csv("./Resources/datasets/origin/credit_card_balance.csv")

installments = pd.read_csv("./Resources/datasets/origin/installments_payments.csv")

cash = pd.read_csv("./Resources/datasets/origin/POS_CASH_balance.csv")

prev_app = pd.read_csv("./Resources/datasets/origin/previous_application.csv")

___
### 1. EXPLOITATION DU MODÈLE : PRÉDICTIONS SUR DE NOUVELLES DONNÉEs

Afin de démontrer toute la chaîne d'exploitation du modèle, cette partie va utiliser le jeu de données TEST application_test.csv.

Ces données n'ayant été utilisées dans aucune des différentes étapes d'élaboration du modèle, elles permettent de montrer l'utilisation du modèle avec de nouvelles données.

___
#### 1.1 CHARGEMENT DU MODÈLE

In [5]:
model = load('../lgbm_trained_model_whole_dataset.joblib')

In [7]:
model_ready_df = hf.transform_data(app_test, 
                                   bureau,                                   
                                   bureau_balance,
                                   card, 
                                   cash, 
                                   installments,
                                   prev_app)

Original Memory Usage: 0.23 gb.
New Memory Usage: 0.1 gb.
Original Memory Usage: 0.66 gb.
New Memory Usage: 0.25 gb.
There are 0 columns with greater than 90% missing values.
Original Memory Usage: 0.49 gb.
New Memory Usage: 0.16 gb.
There are 6 columns with greater than 90% missing values.
Original Memory Usage: 0.64 gb.
New Memory Usage: 0.29 gb.
There are 0 columns with greater than 90% missing values.
Original Memory Usage: 0.71 gb.
New Memory Usage: 0.34 gb.
There are 0 columns with greater than 90% missing values.
Original Memory Usage: 0.87 gb.
New Memory Usage: 0.44 gb.
There are 0 columns with greater than 90% missing values.


In [8]:
# Predictions on the test data
credit_score_predictions = model.predict_proba(model_ready_df)[:, 1]

In [9]:
predictions_df = pd.DataFrame(data=credit_score_predictions*100)\
                   .rename(columns={0:"Credit Score"})

In [10]:
predictions_df = pd.concat([app_test["SK_ID_CURR"], predictions_df], axis=1)

## Saving data for visualization in the dashboard

In [13]:
predictions_df.to_csv("./Resources/datasets/assembled/dashboard/predictions_test.csv")